<a href="https://colab.research.google.com/gist/Malvez/0b49aed3ac5910a75d23dc3b5853a577/implementa-o-do-backpropagation-em-python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
class MLP:

    def __init__(self, lr=0.1, num_input=2, num_hidden=2, num_output=1):

        self.lr = lr

        self.W1 = np.random.uniform(size=(num_input, num_hidden))
        self.W2 = np.random.uniform(size=(num_hidden, num_output))

        self.b1 = np.random.uniform(size=(1, num_hidden))
        self.b2 = np.random.uniform(size=(1, num_output))

        self.hidden_ = None
        self.hidden_out = None
        self.output_ = None
        self.y_pred = None

    def forward(self, X):

        self.hidden_ = np.dot(X, self.W1) + self.b1
        self.hidden_out = self._sigmoid(self.hidden_)

        self.output_ = np.dot(self.hidden_out, self.W2) + self.b2
        self.y_pred = self._sigmoid(self.output_)

        return self.y_pred

    def backward(self, X, y):

        error = (y - self.y_pred)

        d_out_final = error * self._sigmoid(self.y_pred, derivate=True)

        d_out = d_out_final * self.W2.T

        d_hidden_out = d_out * self._sigmoid(self.hidden_out, derivate=True)

        grad1 = np.dot(X.T, d_hidden_out)

        grad2 = np.dot(self.hidden_out.T, d_out_final)

        self.W1 += self.lr * grad1
        self.W2 += self.lr * grad2

        self.b1 += np.sum(self.lr * d_hidden_out, axis=0)
        self.b2 += np.sum(self.lr * d_out_final, axis=0)

    def _sigmoid(self, x, derivate=False):
        if derivate:
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

    def classify(self, x):
        x = np.transpose(x)
        if self.forward(x) >= 0.5:
            return 1
        return 0

    def predict(self, X):
        preds = []
        for x in X:
            x = np.transpose(x)
            if self.forward(x) >= 0.5:
                preds.append(1)
            else:
                preds.append(0)
        return preds

    def fit(self, X, y, num_epochs=1):
        for _ in range(num_epochs):
            self.forward(X)
            self.backward(X, y)

In [ ]:
def criaDataset(n=10, centros=3):
  X, y = make_blobs(n_samples=n, centers=centros, cluster_std=0.2)
  y = y.reshape(-1, 1)
  return X, y

In [ ]:
X, y = criaDataset()

In [ ]:
mlp = MLP(lr=0.2, num_input=X.shape[1], num_hidden=5, num_output=1)

mlp.fit(X, y, num_epochs=1500)

y_pred = mlp.predict(X)

print('\nY_pred:', y_pred)


Y_pred: [1, 0, 1, 0, 1, 1, 1, 0, 1, 0]


In [ ]:
cm_mlp = confusion_matrix(y, y_pred)
cm_mlp

array([[4, 0, 0],
       [0, 3, 0],
       [0, 3, 0]])

In [ ]:
print(classification_report(y, y_pred, labels=None))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       0.50      1.00      0.67         3
           2       0.00      0.00      0.00         3

    accuracy                           0.70        10
   macro avg       0.50      0.67      0.56        10
weighted avg       0.55      0.70      0.60        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
mlp_skl = MLPClassifier(max_iter=100)

mlp_skl.fit(X, y)

y_pred_skl = mlp_skl.predict(X)

#print('\nY_pred:', y_pred_skl)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
cm_mlp_skl = confusion_matrix(y, y_pred_skl)
cm_mlp_skl

array([[4, 0, 0],
       [0, 3, 0],
       [0, 0, 3]])

In [ ]:
scores = cross_validate(MLPClassifier(), X, y, cv=4)
scores['test_score'], np.mean(scores['test_score'])

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/l

(array([1., 1., 1., 1.]), 1.0)

In [ ]:
print(classification_report(y, y_pred_skl, labels=None))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         3

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

